In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [4]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

0

In [5]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [])

# Tensors and GPUs

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

# Storage on the GPU

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.5699, 0.7423, 0.0033],
        [0.8250, 0.0961, 0.3876]])

# Copying

In [11]:
Z = X.cuda(1)
print(X)
print(Z)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
Y + Z

In [ ]:
Z.cuda(1) is Z

# Neural Networks and GPUs

In [12]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

/Users/juicydoggo/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [13]:
net(X)

tensor([[0.2732],
        [0.2732]], grad_fn=<AddmmBackward0>)

In [14]:
net[0].weight.data.device

device(type='cpu')

In [15]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model